In [0]:
import numpy 
import pandas 
from keras.models import Sequential 
from keras.layers import Dense 
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.utils import np_utils 
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import KFold 
from sklearn.preprocessing import LabelEncoder 
from sklearn.pipeline import Pipeline
from sklearn import 
from keras import *

In [0]:
# fix random seed for reproducibility 
seed = 7 
numpy.random.seed(seed)

In [0]:
# load dataset 
dataframe = pandas.read_csv("https://raw.githubusercontent.com/jbrownlee/Datasets/master/sonar.csv", header=None) 
dataset = dataframe.values 
X = dataset[:,0:60].astype(float) 
Y = dataset[:,60]

In [65]:
dataframe.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,0.1609,0.1582,0.2238,0.0645,0.0660,0.2273,0.3100,0.2999,0.5078,0.4797,0.5783,0.5071,0.4328,0.5550,0.6711,0.6415,0.7104,0.8080,0.6791,0.3857,0.1307,0.2604,0.5121,0.7547,0.8537,0.8507,0.6692,0.6097,0.4943,0.2744,0.0510,0.2834,0.2825,0.4256,0.2641,0.1386,0.1051,0.1343,0.0383,0.0324,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,0.4918,0.6552,0.6919,0.7797,0.7464,0.9444,1.0000,0.8874,0.8024,0.7818,0.5212,0.4052,0.3957,0.3914,0.3250,0.3200,0.3271,0.2767,0.4423,0.2028,0.3788,0.2947,0.1984,0.2341,0.1306,0.4182,0.3835,0.1057,0.1840,0.1970,0.1674,0.0583,0.1401,0.1628,0.0621,0.0203,0.0530,0.0742,0.0409,0.0061,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,0.6333,0.7060,0.5544,0.5320,0.6479,0.6931,0.6759,0.7551,0.8929,0.8619,0.7974,0.6737,0.4293,0.3648,0.5331,0.2413,0.5070,0.8533,0.6036,0.8514,0.8512,0.5045,0.1862,0.2709,0.4232,0.3043,0.6116,0.6756,0.5375,0.4719,0.4647,0.2587,0.2129,0.2222,0.2111,0.0176,0.1348,0.0744,0.0130,0.0106,0.0033,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R


In [0]:
# encode class values as integers 
encoder = LabelEncoder() 
encoder.fit(Y) 
encoded_Y = encoder.transform(Y)

 ## Deﬁne The Neural Network Model
 4 inputs -> [4 hidden nodes] -> 3 outputs

Note that we use a sigmoid activation function in the output layer. This is to ensure the output values are in the range of 0 and 1 and may be used as predicted probabilities. Finally, the network uses the ecient ADAM gradient descent optimization algorithm with a logarithmic loss function, which is called categorical crossentropy in Keras.


In [83]:
# define baseline model 
def baseline_model(): 
  # create model 
  model = Sequential() 
  model.add(Dense(60, input_dim=60, init= 'normal' , activation= 'relu' )) 
  model.add(Dense(1, init= 'normal' , activation= 'sigmoid' )) 
  # Compile model 
  model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ]) 
  return model

model = Sequential() 
model.add(Dense(60, input_dim=60, init= 'normal' , activation= 'relu' )) 
model.add(Dense(1, init= 'normal' , activation= 'sigmoid' )) 
# Compile model 
model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ]) 


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(60, input_dim=60, activation="relu", kernel_initializer="normal")`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="normal")`
  if sys.path[0] == '':


## Evaluate The Model with k-Fold Cross Validation

We can now evaluate the neural network model on our training data. The scikit-learn library has excellent capability to evaluate models using a suite of techniques. The gold standard for evaluating machine learning models is k-fold cross validation. First we can deﬁne the model 66 evaluation procedure. Here, we set the number of folds to be 10 (an excellent default) and to shuffle the data before partitioning it.



In [0]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

Now we can evaluate our model (estimator) on our dataset (X and dummy y) using a 10-fold cross validation procedure (kfold). Evaluating the model only takes approximately 10 seconds and returns an object that describes the evaluation of the 10 constructed models for each of the splits of the dataset.


In [85]:
model.fit(X,encoded_Y,nb_epoch=100, batch_size=5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/100
208/208 [==============================] - 0s 669us/step - loss: 0.6874 - accuracy: 0.5144
Epoch 2/100
208/208 [==============================] - 0s 216us/step - loss: 0.6731 - accuracy: 0.5769
Epoch 3/100
208/208 [==============================] - 0s 224us/step - loss: 0.6511 - accuracy: 0.6587
Epoch 4/100
208/208 [==============================] - 0s 223us/step - loss: 0.6251 - accuracy: 0.6779
Epoch 5/100
208/208 [==============================] - 0s 214us/step - loss: 0.5989 - accuracy: 0.7115
Epoch 6/100
208/208 [==============================] - 0s 251us/step - loss: 0.5783 - accuracy: 0.7067
Epoch 7/100
208/208 [==============================] - 0s 229us/step - loss: 0.5432 - accuracy: 0.7644
Epoch 8/100
208/208 [==============================] - 0s 232us/step - loss: 0.5234 - accuracy: 0.7837
Epoch 9/100
208/208 [==============================] - 0s 222us/step - loss: 0.4985 - accuracy: 0.7692
Epoch 10/100
208/208 [==============================] - 0s 226us/step - l

In [86]:
scores = model.evaluate(X, encoded_Y) 
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


208/208 [==============================] - 0s 111us/step
accuracy: 94.23%


In [87]:
from sklearn.model_selection import StratifiedKFold
# define 10-fold cross validation test harness 
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed) 
cvscores = [] 
for train, test in kfold.split(X, encoded_Y): 
  model=baseline_model()
  # Fit the model 
  model.fit(X[train], encoded_Y[train], nb_epoch=150, batch_size=10, verbose=0) 
  # evaluate the model 
  scores = model.evaluate(X[test], encoded_Y[test], verbose=0) 
  print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100)) 
  cvscores.append(scores[1] * 100)

print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores))) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(60, input_dim=60, activation="relu", kernel_initializer="normal")`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="normal")`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


accuracy: 90.48%
accuracy: 95.24%
accuracy: 80.95%
accuracy: 71.43%
accuracy: 85.71%
accuracy: 80.95%
accuracy: 71.43%
accuracy: 71.43%
accuracy: 80.00%
accuracy: 70.00%
79.76% (+/- 8.34%)
